In [1]:
import sys
import torch
import tqdm
import numpy as np
import random
import os
import json
sys.path.append('../../')

In [2]:
from models.cnn.search_cnn import  SearchCNN, SearchCNNController
from models.cnn_darts_hypernet.search_cnn_darts_hypernet_legacy import  SearchCNNControllerWithHyperNet

from configobj import ConfigObj

In [3]:
import utils
# get data with meta info
input_size, input_channels, n_classes, train_data, valid_data = utils.get_data(
    'fashionmnist', '../../data/', cutout_length=0, validation=True)

train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=64,
                                               shuffle=True,
                                               num_workers=32,
                                               pin_memory=True)

valid_loader = torch.utils.data.DataLoader(valid_data,
                                           batch_size=64,
                                           shuffle=False,
                                           num_workers=1,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [4]:

basecfg_path = '../../configs/hyper/fmnist_hyper.cfg'  #конфиг, на который мы ориентируемся при загрузки модели



cfg = ConfigObj(basecfg_path)
name = cfg['name'] # имя для сохранения результатов
ckp_path = 'checkpoint_0_49.ckp' # это шаблон названия сохраненных моделей
seeds = ['0']  # сиды. можно брать из конфига


fine_tune = True # надо ли дообучать модели


In [5]:
def fix_structure(sc): # во время тестов нашей модели нужно перейти от непрерывной структуры к one-hot
    for alpha in sc.alpha_reduce:
        alpha.requires_grad = False
        for subalpha in alpha:
            
            argm = torch.argmax(subalpha)
            subalpha.data*=0
            subalpha.data[argm] += 1
            
            
            

In [6]:
cfg['model_class'] = 'models.cnn_darts_hypernet.search_cnn_darts_hypernet_legacy.SearchCNNControllerWithHyperNet'

In [9]:
sc.net.linear2

Linear(in_features=64, out_features=10, bias=True)

In [8]:
# смотрим качество модели, которое мы получили на обучении, без фиксации структуры
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cuda'
for lam_ in [0.0, 1.0]:
    lam = torch.tensor([lam_]).to('cuda')
    for s in seeds:
        print ('seed {}, lam: {}'.format(s, lam_))
        sc = SearchCNNControllerWithHyperNet(**cfg)
        
        sc.load_state_dict(torch.load(ckp_path.format(s)))
        
        sc = sc.to('cuda')
        sc.eval()
        correct = 0
        total = 0
        for x,y in tqdm.tqdm(valid_loader):
            x = x.cuda()
            y = y.cuda()
            out = sc(x, lam)
            correct += torch.eq(torch.argmax(out, 1), y).sum()
            total += len(x)
        print (correct*1.0/total*1.0)

  0%|          | 0/157 [00:00<?, ?it/s]

seed 0, lam: 0.0


  0%|          | 0/157 [00:00<?, ?it/s]

tensor(0.8205, device='cuda:0')
seed 0, lam: 1.0


100%|██████████| 157/157 [00:03<00:00, 44.11it/s]

tensor(0.7510, device='cuda:0')


In [7]:
out = (sc(x, 0.0*torch.ones(1,1).cuda()) )

NameError: name 'sc' is not defined

In [8]:
torch.eq(torch.argmax(out, 1), y).sum()

NameError: name 'out' is not defined

In [10]:
# смотрим качество модели, которое мы получили на обучении, без фиксации структуры, но с переводом на обычный DARTS
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cuda'
for lam_ in [0.0, 1.0]:
    lam = torch.tensor([lam_])
    for s in seeds:
        print ('seed {}, lam: {}'.format(s, lam_))
        sc0 = SearchCNNControllerWithHyperNet(**cfg)        
        sc0.load_state_dict(torch.load(ckp_path.format(s)))     
        
        sc = SearchCNNController(**cfg)

        sc.net = sc0.net
        for h,a in zip(sc0.hyper_reduce, sc.alpha_reduce):
            a.data = torch.clone(sc0.norm_lam(h(lam)) )
        fix_structure(sc)
        sc = sc.to('cuda')
        sc.sampling_mode='naive'
        sc.eval()
        correct = 0
        total = 0
        for x,y in tqdm.tqdm(valid_loader):
            x = x.cuda()
            y = y.cuda()
            out = sc(x)
            correct += torch.eq(torch.argmax(out, 1), y).sum()
            total += len(x)
        print (correct*1.0/total*1.0)

seed 0, lam: 0.0


AttributeError: 'SearchCNNControllerWithHyperNet' object has no attribute 'norm_lam'

In [17]:
cfg

ConfigObj({'dataset': 'mnist', 'model_class': 'models.cnn_darts_hypernet.search_cnn_darts_hypernet.SearchCNNControllerWithHyperNet', 'name': 'mnist_darts_hypernet_gs', 'batch_size': '64', 'epochs': '50', 'validate_split': '0.5', 'quality': 'top1', 'use_train_quality': '0', 'seeds': '0', 'device': 'cuda', 'darts': {'layers': '1', 'init_channels': '8', 'input_channels': '1', 'n_classes': '10', 'n_nodes': '1', 'stem_multiplier': '3', 'sampling_mode': 'gumbel-softmax', 'initial temp': '1.0', 'delta': '-0.016', 'primitives': 'DARTS', 'optim': {'simple_alpha_update': '1', 'alpha_lr': '0.001', 'w_lr': '0.025', 'w_lr_min': '0.001', 'w_momentum': '0.9', 'w_weight_decay': '0.0', 'w_grad_clip': '5.0', 'alpha_weight_decay': '0.001'}}, 'hypernetwork': {'hidden_layer_num': '0', 'hidden_layer_size': '0', 'log10_lambda_min': '-8', 'log10_lambda_max': '-1', 'lambda sample num': '3'}})

In [9]:
sc0.t = 0.2
sc0 = sc0.cuda()

NameError: name 'sc0' is not defined

In [10]:
(sc(x) - sc0(x, torch.tensor([10e-9]).cuda()))

tensor([[ 2.2452e+00, -4.1482e-01, -9.1407e-01, -9.3603e-01,  4.8313e-01,
          1.2585e+00,  3.9381e+00, -1.2690e+00, -2.0555e+00, -2.2790e+00],
        [-9.5342e-01,  1.5255e+00, -1.3422e-02, -4.6181e-01,  6.5338e-01,
          1.5822e-01, -6.2305e-02,  1.6915e+00, -2.1149e+00, -7.6552e-01],
        [-8.4605e-01, -1.7980e+00,  1.7671e+00,  1.1048e+00, -6.6364e-01,
          1.0337e+00, -6.7482e-01, -1.0612e+00,  1.4877e+00, -2.2239e-01],
        [-2.4437e+00, -2.2934e+00, -1.3324e+00,  8.8219e-01,  1.3497e+00,
          1.4288e+00, -1.4729e+00,  2.6151e+00, -4.8652e-01,  1.1727e+00],
        [ 3.1879e-02, -1.0538e+00,  7.7814e-01, -1.5051e-01, -3.1145e-01,
          1.9647e+00,  5.5327e-01, -1.9894e-01, -2.8079e-01, -1.6805e+00],
        [ 3.9937e+00,  7.6366e+00, -4.8666e+00, -6.4957e+00,  1.0835e+00,
         -6.5719e-01,  4.8326e+00,  6.0533e-01, -3.4464e+00, -3.2394e-01],
        [-1.6168e+00,  7.7880e-01,  1.9563e+00,  1.8378e+00, -4.6235e-01,
          6.9982e-01, -1.5174e+0

In [18]:
sc0.hyper_reduce[0](torch.tensor([0.0])), sc0.hyper_reduce[0](torch.tensor([1.0]))

(tensor([[-0.8079, -0.9030, -0.6230, -0.2134,  3.2586, -0.3532, -0.4448, -0.4817],
         [-0.8494, -0.8544, -0.6758, -0.5583,  4.6323, -0.5762, -0.6024, -0.5743]],
        grad_fn=<ViewBackward>),
 tensor([[-1.0875, -1.2262, -0.8786, -0.3082,  4.5715, -0.4987, -0.6206, -0.6553],
         [-1.1367, -1.1572, -0.9096, -0.7862,  6.3077, -0.7849, -0.8311, -0.7623]],
        grad_fn=<ViewBackward>))

In [19]:
sc0.hyper_reduce[0].model[0].weight, sc0.hyper_reduce[0].model[0].bias

(Parameter containing:
 tensor([[-0.2796],
         [-0.3232],
         [-0.2557],
         [-0.0948],
         [ 1.3129],
         [-0.1455],
         [-0.1758],
         [-0.1736],
         [-0.2873],
         [-0.3027],
         [-0.2337],
         [-0.2279],
         [ 1.6754],
         [-0.2087],
         [-0.2287],
         [-0.1880]], requires_grad=True), Parameter containing:
 tensor([-0.8079, -0.9030, -0.6230, -0.2134,  3.2586, -0.3532, -0.4448, -0.4817,
         -0.8494, -0.8544, -0.6758, -0.5583,  4.6323, -0.5762, -0.6024, -0.5743],
        requires_grad=True))

In [11]:
lam_ = 0.0
for h,a in zip(sc0.hyper_reduce, sc.alpha_reduce):
                a.data = torch.clone(h(torch.tensor([lam_]))) 
       
fix_structure(sc)
for a in sc.alpha_reduce:
    print (a)

Parameter containing:
tensor([[0., 1., -0., -0., -0., -0., -0., 0.],
        [0., 1., -0., -0., -0., -0., -0., 0.]])
Parameter containing:
tensor([[0., 1., -0., -0., -0., -0., -0., 0.],
        [1., 0., -0., -0., -0., -0., -0., 0.],
        [0., 1., 0., -0., -0., -0., -0., 0.]])
Parameter containing:
tensor([[0., 0., -0., -0., -0., -0., -0., 1.],
        [1., 0., -0., -0., -0., -0., -0., 0.],
        [0., 0., 1., -0., -0., -0., -0., 0.],
        [0., 1., 0., -0., -0., -0., -0., 0.]])
Parameter containing:
tensor([[0., 0., -0., -0., -0., -0., -0., 1.],
        [0., 0., -0., -0., -0., -0., -0., 1.],
        [1., -0., -0., -0., -0., -0., -0., -0.],
        [1., -0., -0., -0., -0., -0., -0., -0.],
        [1., -0., -0., -0., -0., -0., -0., -0.]])


In [12]:
lam_ = 1.0
for h,a in zip(sc0.hyper_reduce, sc.alpha_reduce):
                a.data = torch.clone(h(torch.tensor([lam_])) )
       
fix_structure(sc)
for a in sc.alpha_reduce:
    print (a)

Parameter containing:
tensor([[0., 1., -0., -0., -0., -0., -0., 0.],
        [0., 1., -0., -0., -0., -0., -0., 0.]])
Parameter containing:
tensor([[0., 1., -0., -0., -0., -0., -0., 0.],
        [1., 0., -0., -0., -0., -0., -0., 0.],
        [0., 1., 0., -0., -0., -0., -0., 0.]])
Parameter containing:
tensor([[0., 0., -0., -0., -0., -0., -0., 1.],
        [1., 0., -0., -0., -0., -0., -0., 0.],
        [0., 0., 1., -0., -0., -0., -0., 0.],
        [0., 1., 0., -0., -0., -0., -0., 0.]])
Parameter containing:
tensor([[0., 0., -0., -0., -0., -0., -0., 1.],
        [0., 0., -0., -0., -0., -0., -0., 1.],
        [1., -0., -0., -0., -0., -0., -0., -0.],
        [1., 0., -0., -0., -0., -0., -0., -0.],
        [1., -0., -0., -0., -0., -0., -0., -0.]])


In [20]:
import torch.nn.functional as F
def hyperloss(self, X, y, lam):
        #logits = self.forward(X, lam)
        penalty = 0
        
        for id, cell in enumerate(self.net.cells):
            # можно не пробегать несколько раз, т.к. клетки одинаковы (С точностью до normal и reduce)            
            
            lam_ = lam
            weights = [alpha(lam_) for alpha in self.hyper_reduce] if cell.reduction else [
                alpha(lam_) for alpha in self.hyper_normal]
            
            weights = [F.softmax(w/self.t, dim=-1) for w in weights]
            
              
            for edges, w_list in zip(cell.dag, weights):
                for mixed_op, weights in zip(edges, w_list):
                    for op, w in zip(mixed_op._ops, weights):                        
                        for param in op.parameters():
                            penalty += w*np.prod(param.shape)          
            #penalty += lam_[0,0] * (torch.norm(self.net.linear.weight)**2 + torch.norm(self.net.linear.bias)**2)
            #penalty += (1.0-lam_[0,0]) * (torch.norm(self.net.linear2.weight)**2 + torch.norm(self.net.linear2.bias)**2)
            

        # oleg return self.criterion(logits, y)   + penalty * lam[0,0] 
        return penalty 
    

In [18]:
sc0 = sc0.cuda()
sc0.t = 0.1

In [25]:
for r in range(-10, 10):
    
    print (hyperloss(sc0, 0, 0, torch.tensor([[(1.0*r)]]).cuda()))

tensor(18132.4863, device='cuda:0', grad_fn=<AddBackward0>)
tensor(18016.2988, device='cuda:0', grad_fn=<AddBackward0>)
tensor(17835.0156, device='cuda:0', grad_fn=<AddBackward0>)
tensor(17536.4844, device='cuda:0', grad_fn=<AddBackward0>)
tensor(16626.7168, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13917.3027, device='cuda:0', grad_fn=<AddBackward0>)
tensor(12224.7021, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5902.5869, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2624., device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3431, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4539e-18, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5506e-31, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1524e-42, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
tensor(0., device='cud

In [53]:
cfg

ConfigObj({'dataset': 'mnist', 'model_class': 'models.cnn_darts_hypernet.search_cnn_darts_hypernet.SearchCNNControllerWithHyperNet', 'name': 'mnist_darts_hypernet_gs', 'batch_size': '64', 'epochs': '50', 'validate_split': '0.5', 'quality': 'top1', 'use_train_quality': '0', 'seeds': '0', 'device': 'cuda', 'darts': {'layers': '1', 'init_channels': '8', 'input_channels': '1', 'n_classes': '10', 'n_nodes': '1', 'stem_multiplier': '3', 'sampling_mode': 'gumbel-softmax', 'initial temp': '1.0', 'delta': '-0.016', 'primitives': 'DARTS', 'optim': {'simple_alpha_update': '1', 'alpha_lr': '0.001', 'w_lr': '0.025', 'w_lr_min': '0.001', 'w_momentum': '0.9', 'w_weight_decay': '0.0', 'w_grad_clip': '5.0', 'alpha_weight_decay': '0.001'}}, 'hypernetwork': {'hidden_layer_num': '0', 'hidden_layer_size': '0', 'log10_lambda_min': '-8', 'log10_lambda_max': '-1', 'lambda sample num': '3'}})

In [14]:
sc0.hyper_reduce[0].model[0].weight, sc0.hyper_reduce[0].model[0].bias

(Parameter containing:
 tensor([[ 1.1084],
         [-0.0756],
         [-0.0762],
         [-0.0475],
         [-0.6884],
         [-0.0391],
         [-0.0297],
         [-0.0831],
         [ 1.0998],
         [-0.0604],
         [-0.0712],
         [-0.0253],
         [-0.6183],
         [-0.0505],
         [-0.0459],
         [-0.0997]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([ 1.5646, -0.8272, -0.6284, -0.3190,  1.9144, -0.6870, -0.3635, -1.1838,
          1.3121, -0.8585, -0.7832,  0.6159,  1.7367, -0.8624, -0.6465, -1.3189],
        device='cuda:0', requires_grad=True))

In [29]:
for lam_ in [-4.0,  -1.0]:
        lam = torch.tensor([10**lam_])
        # создаем модель с обязательным указанием, что к структуре не применяется softmax
        cfg = ConfigObj(basecfg_path)
        cfg['darts']['sampling_mode'] = 'naive' 
        cfg['device'] = 'cuda'    
        for s in seeds:
            print (s)
            sc0 = SearchCNNControllerWithHyperNet(**cfg)        
            sc0.load_state_dict(torch.load(ckp_path.format(s)))                            
            sc = SearchCNNController(**cfg)
            
            sc.net.load_state_dict(sc0.net.state_dict())
            for h,a in zip(sc0.hyper_reduce, sc.alpha_reduce):
                a.data = torch.clone(h(lam))                                            
            fix_structure(sc)
            
            sc = sc.to('cuda')
            optim = torch.optim.Adam(sc.weights())
            correct = 0
            total = 0
            # дообучаем n эпох
            print ('seed {}, lam: {}'.format(s, lam_))
            for e in range(5):
                for x,y in tqdm.tqdm(train_loader):
                    x = x.cuda()
                    y = y.cuda()            
                    optim.zero_grad()
                    loss = sc.loss(x,y)
                    loss.backward()
                    optim.step()                              
                
                sc.eval()
                correct = 0
                total = 0
                for x,y in tqdm.tqdm(valid_loader):
                    x = x.cuda()
                    y = y.cuda()
                    out = sc(x)
                    correct += torch.eq(torch.argmax(out, 1), y).sum()
                    total += len(x)
                print (correct*1.0/total*1.0)
                sc.train()
            torch.save(sc.state_dict(), ckp_path.format(s)+'{}.fine'.format(lam_))

0


  0%|          | 0/938 [00:00<?, ?it/s]

seed 0, lam: -4.0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  3%|▎         | 27/938 [00:02<01:27, 10.43it/s]


KeyboardInterrupt: 

In [34]:
# смотрим качество модели, которое мы получили на обучении
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cuda'
for lam_ in [-4.0,  -1.0]:
    for s in seeds:
        print ('seed {}, lam: {}'.format(s, lam_))
        sc = SearchCNNController(**cfg)
        
        sc.load_state_dict(torch.load(ckp_path.format(s)+'{}.fine'.format(lam_)))
        
        sc = sc.to('cuda')
        sc.eval()
        correct = 0
        total = 0
        for x,y in tqdm.tqdm(valid_loader):
            x = x.cuda()
            y = y.cuda()
            out = sc(x)
            correct += torch.eq(torch.argmax(out, 1), y).sum()
            total += len(x)
        print (correct*1.0/total*1.0)

  0%|          | 0/157 [00:00<?, ?it/s]

seed 21, lam: -4.0


  0%|          | 0/157 [00:00<?, ?it/s]

tensor(0.0960, device='cuda:0')
seed 21, lam: -1.0


100%|██████████| 157/157 [00:03<00:00, 44.67it/s]

tensor(0.0967, device='cuda:0')


ParameterList(
    (0): Parameter containing: [torch.FloatTensor of size 2x8]
    (1): Parameter containing: [torch.FloatTensor of size 3x8]
    (2): Parameter containing: [torch.FloatTensor of size 4x8]
    (3): Parameter containing: [torch.FloatTensor of size 5x8]
)